<a href="https://colab.research.google.com/github/pj747/qml-experiments/blob/main/QML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install wandb
!wandb login


     |████████████████████████████████| 1.8MB 3.0MB/s 
     |████████████████████████████████| 174kB 18.2MB/s 
     |████████████████████████████████| 133kB 17.7MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7f75d2949c3760dc1f481d168997be6420bd459e4365415eadb02439c5caca97
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=0de46e59da9c22dbf45e2efb5c3ccd76df2d321f30f691c5989223577547cb99
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Ap

In [2]:
!pip install pennylane --upgrade
!pip install pennylane-qulacs["gpu"] --upgrade

     |████████████████████████████████| 460kB 3.0MB/s 
     |████████████████████████████████| 215kB 2.8MB/s 
  Created wheel for qulacs-gpu: filename=Qulacs_GPU-0.2.0-cp37-cp37m-linux_x86_64.whl size=884821 sha256=208ee947821822891797f060defc4aaf32aebbad907140e85b78b7e2019a9d94
  Stored in directory: /root/.cache/pip/wheels/dc/c6/90/6f34fb74a323135ff90fa1a844f9fd5cbc5c6d98edaf9c347f
Successfully built qulacs-gpu


In [3]:
import pennylane as qml

In [4]:

import math
import wandb
from pennylane import numpy as np
from types import SimpleNamespace
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [5]:
dataSet = load_breast_cancer()
X = dataSet.data
Y = dataSet.target
Y = Y * 2 - np.ones(len(Y))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [10]:
def accuracy(labels, predictions):
    accuracy = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            accuracy = accuracy + 1
    accuracy = accuracy / len(labels)

    return accuracy

def lossFunction(labels, predictedLabels):
    loss = 0
    for l, p in zip(labels, predictedLabels):
        loss += (l-p) ** 2
    loss /= len(labels)
    return loss

def cost(qcircuit, params, X, Y):
    predictions = [getPrediction(qcircuit, params, x) for x in X]
    return lossFunction(Y, predictions)

def getPrediction(qcircuit, params, data=None):
    
    quantumOutput = qcircuit(params, data)
    # print("For", data)
    # print("Prediction: ", quantumOutput)
    return quantumOutput
    #no bias

def makeModel(config):
    numQubits = config.numQubits
    dev = qml.device("qulacs.simulator", wires=numQubits, shots=1000, gpu=True)
    @qml.qnode(dev)
    def qcircuit(param, data):
        for i in range(config.numLayers):
            embeddingCircuit(config, data)
            if config.fullEntangle:
                for j in range(numQubits):
                    for i in range(j):
                        qml.CZ(wires=[j,i])
            else:
                for j in range(numQubits-1):
                    qml.CZ(wires=[j,j+1])
            for j in range(numQubits):
                qml.Rot(param[j][i][0], param[j][i][1], param[j][i][2], wires = [j])
            
        return qml.expval(qml.PauliZ(0))

    def embeddingCircuit(config, data):
        norm = np.linalg.norm(data)
        norm = norm if norm !=0 else 1 
        for i in range(0, len(data)-numQubits, numQubits):
            for j in range(numQubits):
                qml.RX(data[i+j]*2*math.pi/norm, wires=j)
    
    return qcircuit


def createAndTrain(config, WandB = False):
    varInit = 0.01 * np.random.randn(config.numQubits, config.numLayers, 3)
    opt = qml.AdamOptimizer()
    batchSize = config.batchSize
    var = varInit
    qcircuit = makeModel(config)
    qcircuit(var, X_train[0])
    circ = qcircuit.draw()
    if WandB:
        wandb.log({"circuit": str(circ)})

    print("Sample Circuit:\n" , circ)
    numBatches = config.numBatches
    for it in range(numBatches):
        batchIndex = np.random.randint(0, len(X_train), (batchSize,))
        X_batch = X_train[batchIndex]
        Y_batch = Y_train[batchIndex]
        var = opt.step(lambda v: cost(qcircuit, v, X_batch, Y_batch), var)
        # Compute accuracy
        print("Computed batch ", it, var)
        predictions = []
        if (it%10 == 0):
            for x in X_test:
                #print(var, x)
                op = qcircuit(var, x)
                predictions.append(np.sign(op))

                #print(qcircuit.draw())
            acc = accuracy(Y_test, predictions)
            loss = lossFunction(Y_test, predictions)
            test_loss = float(loss)
            test_acc = float(acc)
            predictions = []
            if (it%50==0):
                for x in X_train:
                    op = qcircuit(var, x)
                    predictions.append(np.sign(op))
                acc = accuracy(Y_train, predictions)
                loss = lossFunction(Y_train, predictions)
                loss = float(loss)
                a = float(acc)
                if WandB:
                    wandb.log({"train_cost":loss, "train_acc":a}, commit=False)
            if WandB:
                wandb.log({"test_cost":test_loss, "test_acc":test_acc})
    if WandB:
        wandb.log({"parameters": var})
    wandb.finish()

In [11]:

def wandbRun(config):
    wandb.init(project='qml-experiments', entity='pj747', config=config)
    createAndTrain(config, WandB=True)

def wandbSweep():
    run = wandb.init()
    config = run.config
    createAndTrain(config, WandB=True)

In [ ]:
wandb.agent('at9tw1qm', function=wandbSweep, entity = 'pj747', project='qml-experiments')

wandb: Agent Starting Run: w42bhx97 with config:
wandb: 	batchSize: 16
wandb: 	dev: default.qubit
wandb: 	fullEntangle: False
wandb: 	numBatches: 400
wandb: 	numLayers: 1
wandb: 	numQubits: 2


wandb: WARNING Config item 'dev' was locked by 'sweep' (ignored update).


Sample Circuit:
  0: ──RX(0.0709)──RX(0.456)──RX(0.000336)──RX(0.000302)──RX(0.000584)──RX(0.00124)──RX(0.00891)──RX(2.85e-05)──RX(9.89e-05)──RX(7.28e-05)──RX(0.0753)──RX(0.486)──RX(0.000469)──RX(0.000833)──╭C──RZ(-0.00309)──RY(0.00501)──RZ(-0.00834)──┤ ⟨Z⟩ 
 1: ──RX(0.0753)──RX(4.16)───RX(0.000222)──RX(0.00022)───RX(0.000188)──RX(0.00312)──RX(0.139)────RX(4.2e-05)───RX(5.52e-05)──RX(6.32e-06)──RX(0.0916)──RX(4.66)───RX(0.000437)──RX(0.000488)──╰Z──RZ(0.00967)───RY(-0.0136)──RZ(0.0138)────┤     

Computed batch  0 [[[-0.0130873   0.01501338  0.00166207]]

 [[ 0.0096653  -0.01362804  0.01380722]]]
Computed batch  1 [[[-0.01125778  0.02276141  0.00804505]]

 [[ 0.0096653  -0.01362804  0.01380722]]]
Computed batch  2 [[[-0.01011248  0.02989433  0.01258423]]

 [[ 0.0096653  -0.01362804  0.01380722]]]
Computed batch  3 [[[-0.00590558  0.03731458  0.016072  ]]

 [[ 0.0096653  -0.01362804  0.01380722]]]
Computed batch  4 [[[-0.0053497   0.04560096  0.01822845]]

 [[ 0.0096653  -0.01362804  0.